In [1]:
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as skpreprocessing
from sklearn.preprocessing import StandardScaler
import mlflow
from mlflow.models.signature import infer_signature
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import warnings
warnings.filterwarnings("ignore")
import requests, argparse
requests.packages.urllib3.disable_warnings()

### MACROS

In [2]:
NUM_EPOCHS = int(os.getenv("EPOCHS", 10))

LEARNING_RATE = float(os.getenv("LEARNING_RATE", 0.01))

MLFLOW_EXPERIMENT_NAME = os.getenv('DKUBE_PROJECT_NAME')

#DATASET_URL could be specified as Environment parameters at the time of creating JL or Run

# Define data
INPUT_DATA_URL = os.getenv("DATASET_URL", "https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv")


# Keep track of models.
OUTPUT_MODEL_DIR = os.getcwd()+"/model_mlflow"


## create OUTPUT_MODEL_DIR
if not os.path.exists(OUTPUT_MODEL_DIR):
    os.makedirs(OUTPUT_MODEL_DIR)

#### MLFLOW TRACKING INITIALIZATION

In [3]:
if MLFLOW_EXPERIMENT_NAME:
    exp = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
    if not exp:
        print("Creating experiment...")
        mlflow.create_experiment(MLFLOW_EXPERIMENT_NAME)
    mlflow.set_experiment(experiment_name=MLFLOW_EXPERIMENT_NAME)

In [5]:
data = pd.read_csv('/input/dataset/insurance.csv')
insurance_input = data.drop(['charges','timestamp','unique_id'],axis=1)
insurance_target = data['charges']
    
for col in ['sex', 'smoker', 'region']:
    if (insurance_input[col].dtype == 'object'):
        le = skpreprocessing.LabelEncoder()
        le = le.fit(insurance_input[col])
        insurance_input[col] = le.transform(insurance_input[col])
        print('Completed Label encoding on',col)
        
#standardize data
x_scaled = StandardScaler().fit_transform(insurance_input)
x_train, x_test, y_train, y_test = train_test_split(x_scaled,
                                                    insurance_target.values,
                                                    test_size = 0.25,
                                                    random_state=1211)

      Unnamed: 0     sex     bmi  children smoker     region       charges  \
0              0  female  25.900       1.0     no  southwest   5472.449000   
1              1  female  34.105       3.0    yes  northwest  39983.425950   
2              2    male  18.335       0.0     no  northeast  11534.872650   
3              3    male  26.125       0.0     no  northeast   7961.535122   
4              4    male  37.000       0.0     no  southwest   8798.593000   
...          ...     ...     ...       ...    ...        ...           ...   
2403        2403  female  25.740       0.0     no  southeast   3756.621600   
2404        2404    male  31.570       3.0     no  southeast   4837.582300   
2405        2405    male  30.780       2.0     no  northeast   3162.932961   
2406        2406    male  28.975       3.0    yes  northeast  21897.159288   
2407        2407    male  35.200       1.0     no  southwest   2416.955000   

                         timestamp                             

In [6]:
tf.random.set_seed(42)  #first we set random seed
model = keras.Sequential([
      layers.InputLayer(input_shape=(6)),
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

model.compile(loss='mean_absolute_error',
            optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE))

In [7]:
# mlflow metric logging
class loggingCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        mlflow.log_metric("train_loss", logs["loss"], step=epoch)
        mlflow.log_metric("val_loss", logs["val_loss"], step=epoch)
        # output accuracy metric for katib to collect from stdout
        print(f"loss={round(logs['loss'],2)}")

#### ML TRAINING

In [8]:
with mlflow.start_run(run_name="insurance") as run:
    
    model.fit(x_train, y_train, epochs = NUM_EPOCHS, verbose=0,
                validation_split=0.1, callbacks=[loggingCallback()])
    
    # Exporting model
    model.save(filepath=os.path.join(OUTPUT_MODEL_DIR, '1'))
    
    # Two ways to save model - log_artifacts() or log_model()
    mlflow.log_artifacts(OUTPUT_MODEL_DIR) ## For tf-serving
    signature = infer_signature(x_test, model.predict(x_test))
    mlflow.keras.log_model(keras_model=model, artifact_path=None, signature=signature)
        
    # Record parameters
    mlflow.log_params({"dataset": '/input/data/insurance.csv',
                       "code": "https://github.com/oneconvergence/dkube-examples/tree/training/insurance",
                       "model": "Deep Neural Network"})
    
print("Training Complete !")

loss=13196.65
loss=10517.24
loss=7761.93
loss=7011.79
loss=6878.52
loss=6823.34
loss=6794.45
loss=6775.81
loss=6763.17
loss=6740.66
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /home/larryc1200/workspace/ins-larryc1200-pre/insurance/model_mlflow/1/assets
INFO:tensorflow:Assets written to: /tmp/tmpz1q4oklw/model/data/model/assets
Training Complete !
